<a href="https://colab.research.google.com/github/InnovateWithAlex/project-planner/blob/main/Project_PlannerGC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary packages
!pip install matplotlib pandas ipywidgets plotly --upgrade

# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import plotly.express as px
from IPython.display import display, clear_output

# Step 1: Create an empty DataFrame for Projects
project_data = pd.DataFrame(columns=[
    "Project Name", "Project Start Date", "Project End Date", "Project Status"
])

# Function to collect project information from the user
def add_project():
    global project_data
    clear_output()

    # Collect Project Information
    print("Please enter the project information below:")
    project_name = input("Project Name (leave blank for default): ").strip() or f"Unnamed Project {len(project_data) + 1}"
    project_start_date = widgets.DatePicker(description="Project Start Date:")
    project_end_date = widgets.DatePicker(description="Project End Date:")
    project_status = widgets.Dropdown(
        options=["Not Started", "In Progress", "In Review", "Pass", "Fail", "Pushed"],
        description="Project Status:"
    )
    num_tasks = widgets.IntSlider(value=5, min=1, max=10, step=1, description="Number of Tasks:")

    # Display widgets for project information
    display(project_start_date, project_end_date, project_status, num_tasks)

    # Button to submit project details
    submit_project_button = widgets.Button(description="Submit Project")

    def on_submit_project(b):
        # Collect Tasks Information
        clear_output()
        tasks = []
        for i in range(1, num_tasks.value + 1):
            task_start = widgets.DatePicker(description=f"Task {i} Start Date:")
            task_end = widgets.DatePicker(description=f"Task {i} End Date:")
            task_status = widgets.Dropdown(
                options=["Not Started", "In Progress", "In Review", "Pass", "Fail", "Pushed"],
                description=f"Task {i} Status:"
            )
            display(task_start, task_end, task_status)
            tasks.append((task_start, task_end, task_status))

        # Button to submit task details
        submit_tasks_button = widgets.Button(description="Submit Tasks")

        def on_submit_tasks(bt):
            clear_output()
            new_row = {
                "Project Name": project_name,
                "Project Start Date": project_start_date.value,
                "Project End Date": project_end_date.value,
                "Project Status": project_status.value,
            }

            # Adding task details to the dictionary
            for i in range(1, num_tasks.value + 1):
                new_row[f"Task {i} Start"] = tasks[i - 1][0].value
                new_row[f"Task {i} End"] = tasks[i - 1][1].value
                new_row[f"Task {i} Status"] = tasks[i - 1][2].value

            # Append the new row to the DataFrame
            global project_data
            project_data = pd.concat([project_data, pd.DataFrame([new_row])], ignore_index=True)

            # Display updated DataFrame
            display(project_data)

            # Plot projects after adding
            plot_projects()

        submit_tasks_button.on_click(on_submit_tasks)
        display(submit_tasks_button)

    submit_project_button.on_click(on_submit_project)
    display(submit_project_button)

# Call the add_project function to add a new project
add_project()

# Visualization using Plotly
def plot_projects():
    if project_data.empty:
        return

    # Prepare data for plotting using Plotly
    project_data_long = []

    for idx, row in project_data.iterrows():
        # Add the project timeline
        if pd.notna(row['Project Start Date']) and pd.notna(row['Project End Date']):
            project_data_long.append({
                'Task': row['Project Name'],
                'Start': row['Project Start Date'],
                'Finish': row['Project End Date'],
                'Status': row['Project Status']
            })

        # Add the individual tasks timeline
        for i in range(1, 11):
            task_start = row.get(f"Task {i} Start", None)
            task_end = row.get(f"Task {i} End", None)
            task_status = row.get(f"Task {i} Status", "Not Started")

            if pd.notna(task_start) and pd.notna(task_end):
                project_data_long.append({
                    'Task': f"{row['Project Name']} - Task {i}",
                    'Start': task_start,
                    'Finish': task_end,
                    'Status': task_status
                })

    if not project_data_long:
        return

    # Convert to DataFrame
    plot_df = pd.DataFrame(project_data_long)

    # Create Gantt chart using Plotly
    fig = px.timeline(plot_df, x_start='Start', x_end='Finish', y='Task', color='Status',
                      title='Project Timelines', labels={'Task': 'Projects and Tasks'})
    fig.update_yaxes(categoryorder='total ascending')
    fig.update_layout(xaxis_title='Date', yaxis_title='Tasks',
                      xaxis=dict(tickformat='%Y-%m-%d'),
                      template='plotly_white')
    fig.show()

# Call the plot_projects function to visualize the projects
plot_projects()


,Project Name,Project Start Date,Project End Date,Project Status,Task 1 Start,Task 1 End,Task 1 Status,Task 2 Start,Task 2 End,Task 2 Status
0,Alpha1,2024-10-28,2024-11-25,In Progress,2024-10-28,2024-11-04,Pass,2024-11-05,2024-11-25,In Progress


# New Section